지능정보SW아카데미 - SW, AI(빅데이터, 정답) <br>
데이터수집 - 저장/관리 - 분석 - 시각화(SW)<br>
Crawler   DB(RDBMS)  통계/확률/기계학습    앱/웹/익스텐션(크로미움)<br>
         (Hadoop/Spark)+             React, JS+HTML+CSS<br>
          ER Model -> Diagram, SQL(DDL,DML,join)<br>
          특; ER -> Database-Tabls<br>
             기능 -> 절차 많음<br>
          ORM - SQL, 기능개발<br>
        ORM,  Core:RDBMS 통합 관리/연결/SQL해석<br>


In [1]:
# 엔진 만들기
from sqlalchemy import create_engine, MetaData
from sqlalchemy.schema import Table, Column, ForeignKey
from sqlalchemy.types import Integer, Text

In [5]:
# echo를 True로 설정하면 log에 SQL이 기록된다.
# engine을 만드는 순간 실행된다.
engine = create_engine('sqlite:///:memory:', echo = True)

In [6]:
# 사후에 엔진을 바인딩 시킬 수 있다.
meta = MetaData()

In [ ]:
# 기존 SQLites와 다르게 테이블을 구성할 테이블 명과 열을 정의하면 간단하게 DB를 만들 수있다.
user = Table('USER', meta, 
             Column('PK', Integer, primary_key = True),
             Column('NAME', Text, nullable = True))

In [ ]:
meta.create_all(engine)

In [ ]:
# reflect를 사용하면 방금 다 날린 테이블을 다시 불러온다
meta.reflect(engine)

In [ ]:
# SQLites와 비슷하게 insert 구문을 통해 데이터를 넣는데, 
# 객체를 만들었기 때문에 양식에 맞춰서 넣기만 하면 된다.
user1 = user.insert().values(NAME = '아무개')

In [ ]:
# 실행된 결과가 어떻게 컴파일된건지 알기위해서 compile()을 사용한다.
print(user1.compile(), user1.compile().params)

In [3]:
import sqlalchemy
# !pip install sqlalchemy --upgrade
sqlalchemy.__version__

'2.0.20'

In [2]:
from sqlalchemy.sql import select, insert, update, delete, join, func

In [7]:
# SQLite와 같이 엔진도 커서를 지정할 수 있다.
con = engine.connect()
#con.execute('SELECT * FROM ...')

In [ ]:
meta.clear()

In [8]:
meta.tables

FacadeDict({})

In [9]:
Table('USER', meta,
      Column('PK', Integer, primary_key = True),
      Column('NAME', Text, nullable = False))

Table('USER', MetaData(), Column('PK', Integer(), table=<USER>, primary_key=True, nullable=False), Column('NAME', Text(), table=<USER>, nullable=False), schema=None)

In [10]:
len(meta.tables)

1

In [11]:
meta.tables.keys()
User = meta.tables['USER']

In [12]:
print(User.insert())

INSERT INTO "USER" ("PK", "NAME") VALUES (:PK, :NAME)


In [13]:
# DB에 반영
meta.create_all(engine)

2023-09-06 11:23:53,392 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-06 11:23:53,393 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("USER")
2023-09-06 11:23:53,395 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-06 11:23:53,397 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("USER")
2023-09-06 11:23:53,398 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-06 11:23:53,400 INFO sqlalchemy.engine.Engine 
CREATE TABLE "USER" (
	"PK" INTEGER NOT NULL, 
	"NAME" TEXT NOT NULL, 
	PRIMARY KEY ("PK")
)


2023-09-06 11:23:53,401 INFO sqlalchemy.engine.Engine [no key 0.00107s] ()
2023-09-06 11:23:53,402 INFO sqlalchemy.engine.Engine COMMIT


In [ ]:
# 다 날림 (DB에 있는 것도 날림) > 객체로서의 테이블 삭제함.
meta.drop_all(engine)

# drop으로 날리면 table이 사라진다 > 다시 table 만들고 > create_all

In [ ]:
# Table도 삭제함
meta.remove(User)

In [ ]:
meta.tables

In [ ]:
# DB에 있던 Table을 다시 불러옴
meta.reflect(engine)

In [14]:
meta.tables

FacadeDict({'USER': Table('USER', MetaData(), Column('PK', Integer(), table=<USER>, primary_key=True, nullable=False), Column('NAME', Text(), table=<USER>, nullable=False), schema=None)})

In [15]:
print(User.insert().values(PK = 1, NAME = '아무개'))

INSERT INTO "USER" ("PK", "NAME") VALUES (:PK, :NAME)


In [16]:
print(User.insert().values(PK = 1, NAME = '아무개').compile().params)

{'PK': 1, 'NAME': '아무개'}


In [17]:
con.execute(User.insert().values(PK=1, NAME = '아무개'))

2023-09-06 11:24:03,790 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-06 11:24:03,792 INFO sqlalchemy.engine.Engine INSERT INTO "USER" ("PK", "NAME") VALUES (?, ?)
2023-09-06 11:24:03,793 INFO sqlalchemy.engine.Engine [generated in 0.00164s] (1, '아무개')


In [18]:
con.execute(User.select()).fetchall()

2023-09-06 11:24:05,569 INFO sqlalchemy.engine.Engine SELECT "USER"."PK", "USER"."NAME" 
FROM "USER"
2023-09-06 11:24:05,570 INFO sqlalchemy.engine.Engine [generated in 0.00078s] ()


[(1, '아무개')]

In [19]:
Table('ADDRESS', meta,
      Column('PK', Integer, primary_key=True),
      Column('NAME', Text),
      # USER 테이블의 PK값을 참조한다.
      Column('FK', Integer, ForeignKey('USER.PK')))

Table('ADDRESS', MetaData(), Column('PK', Integer(), table=<ADDRESS>, primary_key=True, nullable=False), Column('NAME', Text(), table=<ADDRESS>), Column('FK', Integer(), ForeignKey('USER.PK'), table=<ADDRESS>), schema=None)

In [20]:
Table('TEMP', meta,
      Column('PK', Integer, primary_key=True),
      Column('NAME', Text),
      # USER 테이블의 PK값을 참조한다.
      Column('FK', Integer,nullable=False))

Table('TEMP', MetaData(), Column('PK', Integer(), table=<TEMP>, primary_key=True, nullable=False), Column('NAME', Text(), table=<TEMP>), Column('FK', Integer(), table=<TEMP>, nullable=False), schema=None)

In [21]:
User.columns['PK']

Column('PK', Integer(), table=<USER>, primary_key=True, nullable=False)

In [22]:
# DB에 반영
meta.create_all(engine)

2023-09-06 11:24:13,673 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-06 11:24:13,674 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("USER")
2023-09-06 11:24:13,675 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-06 11:24:13,676 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("ADDRESS")
2023-09-06 11:24:13,676 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-06 11:24:13,677 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("ADDRESS")
2023-09-06 11:24:13,677 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-06 11:24:13,679 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("TEMP")
2023-09-06 11:24:13,679 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-06 11:24:13,680 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("TEMP")
2023-09-06 11:24:13,681 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-06 11:24:13,683 INFO sqlalchemy.engine.Engine 
CREATE TABLE "ADDRESS" (
	"PK" INTEGER NOT NULL, 
	"NAME" TEXT, 
	"FK" INTEGER, 
	PRIMARY KEY ("PK"), 
	FORE

In [23]:
len(meta.tables)

3

In [24]:
# 이 테이블에는 FK를 반영
# 값을 어떻게 넣는지는 print를 통해 확인하면 된다.
# print(meta.tables['ADDRESS'].insert())

con.execute(meta.tables['ADDRESS'].insert().values(PK= 1, NAME = '주소', FK = 1))

2023-09-06 11:24:18,244 INFO sqlalchemy.engine.Engine INSERT INTO "ADDRESS" ("PK", "NAME", "FK") VALUES (?, ?, ?)
2023-09-06 11:24:18,245 INFO sqlalchemy.engine.Engine [generated in 0.00095s] (1, '주소', 1)


In [25]:
# 이 테이블에서는 FK를 주지 않음.
con.execute(meta.tables['TEMP'].insert().values(PK= 1, NAME = '가짜', FK = 1))

2023-09-06 11:24:20,125 INFO sqlalchemy.engine.Engine INSERT INTO "TEMP" ("PK", "NAME", "FK") VALUES (?, ?, ?)
2023-09-06 11:24:20,126 INFO sqlalchemy.engine.Engine [generated in 0.00088s] (1, '가짜', 1)


meta.drop_all(engine); ORM -> DBMS : MetaData에 모든 Table을 삭제한다. <br>
meta.remove(객체); ORM Table 객체 : ORM Table의 객체를 삭제한다. <br>
meta.reflect(engine); DBMS -> ORM : DBMS에 Table이 있는 경우 다시 복구한다.<br>
<br><br>
정리하면, drop_all을 통해 바인딩된 engine은 새로 Table을 다시 commit해야하지만,<br>
remove를 통해 제거한 Table 객체는 reflect를 통해 다시 복구 가능하다.

In [26]:
rst = con.execute(meta.tables['ADDRESS'].select())

for _ in rst:
    print(_)

2023-09-06 11:24:22,631 INFO sqlalchemy.engine.Engine SELECT "ADDRESS"."PK", "ADDRESS"."NAME", "ADDRESS"."FK" 
FROM "ADDRESS"
2023-09-06 11:24:22,632 INFO sqlalchemy.engine.Engine [generated in 0.00068s] ()
(1, '주소', 1)


In [27]:
# Cross join처럼 사용가능하다.
con.execute(select(meta.tables['USER'],
                   meta.tables['ADDRESS'],
                   meta.tables['TEMP'])).fetchall()

2023-09-06 11:24:23,932 INFO sqlalchemy.engine.Engine SELECT "USER"."PK", "USER"."NAME", "ADDRESS"."PK" AS "PK_1", "ADDRESS"."NAME" AS "NAME_1", "ADDRESS"."FK", "TEMP"."PK" AS "PK_2", "TEMP"."NAME" AS "NAME_2", "TEMP"."FK" AS "FK_1" 
FROM "USER", "ADDRESS", "TEMP"
2023-09-06 11:24:23,933 INFO sqlalchemy.engine.Engine [generated in 0.00080s] ()


C:\Users\GyeongJun\AppData\Local\Temp\ipykernel_13968\708192316.py:2: SAWarning: SELECT statement has a cartesian product between FROM element(s) "TEMP", "USER" and FROM element "ADDRESS".  Apply join condition(s) between each element to resolve.
  con.execute(select(meta.tables['USER'],


[(1, '아무개', 1, '주소', 1, 1, '가짜', 1)]

In [28]:
# 또한 columns을 간단하게 c로 하고 . 뒤에 컬럼명을 붙히면 된다.
con.execute(select(meta.tables['USER'].columns['PK'],
                   meta.tables['USER'].c.NAME,
                   meta.tables['USER'].c.PK)).fetchall()

2023-09-06 11:24:25,676 INFO sqlalchemy.engine.Engine SELECT "USER"."PK", "USER"."NAME", "USER"."PK" AS "PK__1" 
FROM "USER"
2023-09-06 11:24:25,677 INFO sqlalchemy.engine.Engine [generated in 0.00085s] ()


[(1, '아무개', 1)]

In [29]:
# Table을 지정한 뒤
USER = meta.tables['USER']
ADDR = meta.tables['ADDRESS']
TEMP = meta.tables['TEMP']

In [30]:
# SQL 구문을 보기위해 작성할 수 있다.
print(USER.select().where(USER.c.PK == 1))

SELECT "USER"."PK", "USER"."NAME" 
FROM "USER" 
WHERE "USER"."PK" = :PK_1


In [31]:
print(USER.c.PK == TEMP.c.FK)

"USER"."PK" = "TEMP"."FK"


In [32]:
# ADDRESS Table은 FK가 있기 때문에 SQL의 ON절을 바로 뽑을 수 있다.
print(USER.join(ADDR))

"USER" JOIN "ADDRESS" ON "USER"."PK" = "ADDRESS"."FK"


In [33]:
# 하지만 TEMP와 같이 FK가 없는 경우 뒤에 ON 절을 기재하면 FK 없이도 SQL의 ON절을 뽑을 수 있다.
print(USER.join(TEMP, USER.c.PK == TEMP.c.FK))
# 실제 SQL문에선 """select * From user Join temp on user.pk = temp.fk""" (inner join)

"USER" JOIN "TEMP" ON "USER"."PK" = "TEMP"."FK"


In [34]:
from sqlalchemy.sql import or_, and_, between
print(or_(USER.c.PK == 1, USER.c.PK == 2))

"USER"."PK" = :PK_1 OR "USER"."PK" = :PK_2


In [35]:
print(USER.c.NAME.like('%어쩌고%'))

"USER"."NAME" LIKE :NAME_1


In [36]:
print(USER.select().where(USER.c.NAME.like('%어쩌고%')).\
      group_by(USER.c.PK).\
      order_by(USER.c.PK))

SELECT "USER"."PK", "USER"."NAME" 
FROM "USER" 
WHERE "USER"."NAME" LIKE :NAME_1 GROUP BY "USER"."PK" ORDER BY "USER"."PK"


In [37]:
# USER 가 정의되어 있으므로 SQL에 넣을 문자를 간단하게 표현할 수있다.
print((USER.c.NAME == 'abcd') & (USER.c.PK == 1) | (USER.c.PK == 2))

"USER"."NAME" = :NAME_1 AND "USER"."PK" = :PK_1 OR "USER"."PK" = :PK_2


In [38]:
# drop_all로 USER를 날려서 FK정보가 날라감. 다시 테이블 지우고 실행하면 됨.
print(USER.join(ADDR))

"USER" JOIN "ADDRESS" ON "USER"."PK" = "ADDRESS"."FK"


In [39]:
# 엔진 초기화
engine.dispose()

In [40]:
meta.clear()

In [41]:
meta.tables

FacadeDict({})

In [42]:
engine = create_engine('sqlite:///:memory:', echo = True)

In [43]:
# 아티스트
Table('T_A', meta,
      Column('PK', Integer, primary_key= True),
      Column('NAME', Text, nullable= False))
# 앨범 (아티스트의 PK key를 FK키로 받음)
Table('T_B', meta,
      Column('PK', Integer, primary_key= True),
      Column('NAME', Text, nullable= False),
      Column('FK', None, ForeignKey('T_A.PK')))
# 장르
Table('T_C', meta,
      Column('PK', Integer, primary_key= True),
      Column('NAME', Text, nullable= False))
# 곡  (앨범의 PK key를 FK키로 받음)
Table('T_D', meta,
      Column('PK', Integer, primary_key= True),
      Column('NAME', Text, nullable= False),
      Column('FK1', None, ForeignKey('T_B.PK'), nullable=False),
      Column('FK2', Integer, nullable=False))

Table('T_D', MetaData(), Column('PK', Integer(), table=<T_D>, primary_key=True, nullable=False), Column('NAME', Text(), table=<T_D>, nullable=False), Column('FK1', Integer(), ForeignKey('T_B.PK'), table=<T_D>, nullable=False), Column('FK2', Integer(), table=<T_D>, nullable=False), schema=None)

In [44]:
len(meta.tables)

4

In [45]:
meta.create_all(engine)

2023-09-06 11:24:57,161 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-06 11:24:57,162 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("T_A")
2023-09-06 11:24:57,163 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-06 11:24:57,164 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("T_A")
2023-09-06 11:24:57,164 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-06 11:24:57,165 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("T_B")
2023-09-06 11:24:57,165 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-06 11:24:57,166 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("T_B")
2023-09-06 11:24:57,166 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-06 11:24:57,167 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("T_C")
2023-09-06 11:24:57,169 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-06 11:24:57,169 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("T_C")
2023-09-06 11:24:57,170 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-06 11:24:57,170

In [46]:
con = engine.connect()

In [47]:
for name in ['가수1', '가수2', '가수3', '가수4']:
    con.execute(meta.tables['T_A'].insert().values(NAME = name))
    """ INSERT INTO T_A (NAME) VALUES(:name)"""

2023-09-06 11:25:03,521 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-06 11:25:03,522 INFO sqlalchemy.engine.Engine INSERT INTO "T_A" ("NAME") VALUES (?)
2023-09-06 11:25:03,523 INFO sqlalchemy.engine.Engine [generated in 0.00146s] ('가수1',)
2023-09-06 11:25:03,523 INFO sqlalchemy.engine.Engine INSERT INTO "T_A" ("NAME") VALUES (?)
2023-09-06 11:25:03,524 INFO sqlalchemy.engine.Engine [cached since 0.003025s ago] ('가수2',)
2023-09-06 11:25:03,526 INFO sqlalchemy.engine.Engine INSERT INTO "T_A" ("NAME") VALUES (?)
2023-09-06 11:25:03,527 INFO sqlalchemy.engine.Engine [cached since 0.005464s ago] ('가수3',)
2023-09-06 11:25:03,528 INFO sqlalchemy.engine.Engine INSERT INTO "T_A" ("NAME") VALUES (?)
2023-09-06 11:25:03,528 INFO sqlalchemy.engine.Engine [cached since 0.007022s ago] ('가수4',)


In [48]:
A = meta.tables['T_A']
B = meta.tables['T_B']
C = meta.tables['T_C']
D = meta.tables['T_D']

In [49]:
for key in ['1','2','3','4']:
    for name in ['앨범1', '앨범2']:
        f'가수{key}_{name}'

        # 맨 마지막이 key인 A의 PK를 
        rst = con.execute(select(A.c.PK).where(A.c.NAME.like('%'+key)))
        # 하나씩 불러오기
        rst = rst.fetchone()[0]
        # 앨범이름과 참조키 넣기
        con.execute(B.insert().values(NAME = name, FK = rst))

2023-09-06 11:25:34,426 INFO sqlalchemy.engine.Engine SELECT "T_A"."PK" 
FROM "T_A" 
WHERE "T_A"."NAME" LIKE ?
2023-09-06 11:25:34,428 INFO sqlalchemy.engine.Engine [generated in 0.00250s] ('%1',)
2023-09-06 11:25:34,432 INFO sqlalchemy.engine.Engine INSERT INTO "T_B" ("NAME", "FK") VALUES (?, ?)
2023-09-06 11:25:34,433 INFO sqlalchemy.engine.Engine [generated in 0.00127s] ('앨범1', 1)
2023-09-06 11:25:34,434 INFO sqlalchemy.engine.Engine SELECT "T_A"."PK" 
FROM "T_A" 
WHERE "T_A"."NAME" LIKE ?
2023-09-06 11:25:34,434 INFO sqlalchemy.engine.Engine [cached since 0.008489s ago] ('%1',)
2023-09-06 11:25:34,435 INFO sqlalchemy.engine.Engine INSERT INTO "T_B" ("NAME", "FK") VALUES (?, ?)
2023-09-06 11:25:34,436 INFO sqlalchemy.engine.Engine [cached since 0.004617s ago] ('앨범2', 1)
2023-09-06 11:25:34,437 INFO sqlalchemy.engine.Engine SELECT "T_A"."PK" 
FROM "T_A" 
WHERE "T_A"."NAME" LIKE ?
2023-09-06 11:25:34,437 INFO sqlalchemy.engine.Engine [cached since 0.01162s ago] ('%2',)
2023-09-06 11:2

In [51]:
# 실제 적용된 B 테이블 display
rst = con.execute(B.select())
rst.fetchall()

2023-09-06 11:28:16,856 INFO sqlalchemy.engine.Engine SELECT "T_B"."PK", "T_B"."NAME", "T_B"."FK" 
FROM "T_B"
2023-09-06 11:28:16,857 INFO sqlalchemy.engine.Engine [generated in 0.00079s] ()


[(1, '앨범1', 1),
 (2, '앨범2', 1),
 (3, '앨범1', 2),
 (4, '앨범2', 2),
 (5, '앨범1', 3),
 (6, '앨범2', 3),
 (7, '앨범1', 4),
 (8, '앨범2', 4)]

In [53]:
# T_A의 PK와 T_B의 FK가 동일한 T_A의 row뽑기
rst = con.execute(A.select().join(B))
rst.fetchall()

2023-09-06 11:28:58,608 INFO sqlalchemy.engine.Engine SELECT "T_A"."PK", "T_A"."NAME" 
FROM "T_A" JOIN "T_B" ON "T_A"."PK" = "T_B"."FK"
2023-09-06 11:28:58,608 INFO sqlalchemy.engine.Engine [cached since 126.2s ago] ()


[(1, '가수1'),
 (1, '가수1'),
 (2, '가수2'),
 (2, '가수2'),
 (3, '가수3'),
 (3, '가수3'),
 (4, '가수4'),
 (4, '가수4')]

In [54]:
for name in ['장르1','장르2','장르3','장르4']:
    # T_C 테이블에 장르 넣기
    rst = con.execute(meta.tables['T_C'].insert().values(NAME = name))

2023-09-06 11:30:18,954 INFO sqlalchemy.engine.Engine INSERT INTO "T_C" ("NAME") VALUES (?)
2023-09-06 11:30:18,955 INFO sqlalchemy.engine.Engine [generated in 0.00141s] ('장르1',)
2023-09-06 11:30:18,955 INFO sqlalchemy.engine.Engine INSERT INTO "T_C" ("NAME") VALUES (?)
2023-09-06 11:30:18,955 INFO sqlalchemy.engine.Engine [cached since 0.0034s ago] ('장르2',)
2023-09-06 11:30:18,958 INFO sqlalchemy.engine.Engine INSERT INTO "T_C" ("NAME") VALUES (?)
2023-09-06 11:30:18,959 INFO sqlalchemy.engine.Engine [cached since 0.005352s ago] ('장르3',)
2023-09-06 11:30:18,961 INFO sqlalchemy.engine.Engine INSERT INTO "T_C" ("NAME") VALUES (?)
2023-09-06 11:30:18,962 INFO sqlalchemy.engine.Engine [cached since 0.008134s ago] ('장르4',)


In [55]:
# 실제 적용된 C 테이블 display
# PK는 primary_key이므로 값을 따로 설정 안해도 된다.
rst = con.execute(C.select())
rst.fetchall()

2023-09-06 11:30:33,893 INFO sqlalchemy.engine.Engine SELECT "T_C"."PK", "T_C"."NAME" 
FROM "T_C"
2023-09-06 11:30:33,894 INFO sqlalchemy.engine.Engine [generated in 0.00110s] ()


[(1, '장르1'), (2, '장르2'), (3, '장르3'), (4, '장르4')]

In [56]:
# 이제 T_D를 만들기 위해서 상속키를 만들 T_B를 불러온다. (상속관계 : T_B.c.PK > T_D.c.FK)
B_select = con.execute(select(B.c.PK)).fetchall()
for row in B_select:
    for key in ['1','2','3','4']:
        for name in ['노래1', '노래2', '노래3', '노래4']:
            # FK2는 T_B를 만들 때 처럼 T_A의 PK 값을 상속한다.
            rst = con.execute(select(C.c.PK).where(A.c.NAME.like('%'+key)))
            rst = rst.fetchone()[0]     

            con.execute(D.insert().values(NAME = f'{row[0]}앨범_장르{key}_{name}'
                                          , FK1 = row[0], FK2 = rst))

2023-09-06 11:32:06,475 INFO sqlalchemy.engine.Engine SELECT "T_B"."PK" 
FROM "T_B"
2023-09-06 11:32:06,476 INFO sqlalchemy.engine.Engine [generated in 0.00079s] ()
2023-09-06 11:32:06,478 INFO sqlalchemy.engine.Engine SELECT "T_C"."PK" 
FROM "T_C", "T_A" 
WHERE "T_A"."NAME" LIKE ?
2023-09-06 11:32:06,479 INFO sqlalchemy.engine.Engine [generated in 0.00127s] ('%1',)
2023-09-06 11:32:06,482 INFO sqlalchemy.engine.Engine INSERT INTO "T_D" ("NAME", "FK1", "FK2") VALUES (?, ?, ?)
2023-09-06 11:32:06,483 INFO sqlalchemy.engine.Engine [generated in 0.00070s] ('1앨범_장르1_노래1', 1, 1)
2023-09-06 11:32:06,484 INFO sqlalchemy.engine.Engine SELECT "T_C"."PK" 
FROM "T_C", "T_A" 
WHERE "T_A"."NAME" LIKE ?
2023-09-06 11:32:06,485 INFO sqlalchemy.engine.Engine [cached since 0.006571s ago] ('%1',)
2023-09-06 11:32:06,486 INFO sqlalchemy.engine.Engine INSERT INTO "T_D" ("NAME", "FK1", "FK2") VALUES (?, ?, ?)
2023-09-06 11:32:06,486 INFO sqlalchemy.engine.Engine [cached since 0.004306s ago] ('1앨범_장르1_노래2',

C:\Users\GyeongJun\AppData\Local\Temp\ipykernel_13968\527740505.py:6: SAWarning: SELECT statement has a cartesian product between FROM element(s) "T_C" and FROM element "T_A".  Apply join condition(s) between each element to resolve.
  rst = con.execute(select(C.c.PK).where(A.c.NAME.like('%'+key)))


2023-09-06 11:32:06,665 INFO sqlalchemy.engine.Engine SELECT "T_C"."PK" 
FROM "T_C", "T_A" 
WHERE "T_A"."NAME" LIKE ?
2023-09-06 11:32:06,666 INFO sqlalchemy.engine.Engine [cached since 0.1887s ago] ('%4',)
2023-09-06 11:32:06,669 INFO sqlalchemy.engine.Engine INSERT INTO "T_D" ("NAME", "FK1", "FK2") VALUES (?, ?, ?)
2023-09-06 11:32:06,669 INFO sqlalchemy.engine.Engine [cached since 0.1871s ago] ('3앨범_장르4_노래3', 3, 1)
2023-09-06 11:32:06,671 INFO sqlalchemy.engine.Engine SELECT "T_C"."PK" 
FROM "T_C", "T_A" 
WHERE "T_A"."NAME" LIKE ?
2023-09-06 11:32:06,672 INFO sqlalchemy.engine.Engine [cached since 0.1938s ago] ('%4',)
2023-09-06 11:32:06,674 INFO sqlalchemy.engine.Engine INSERT INTO "T_D" ("NAME", "FK1", "FK2") VALUES (?, ?, ?)
2023-09-06 11:32:06,676 INFO sqlalchemy.engine.Engine [cached since 0.1933s ago] ('3앨범_장르4_노래4', 3, 1)
2023-09-06 11:32:06,677 INFO sqlalchemy.engine.Engine SELECT "T_C"."PK" 
FROM "T_C", "T_A" 
WHERE "T_A"."NAME" LIKE ?
2023-09-06 11:32:06,678 INFO sqlalchem

In [57]:
# 집계함수도 사용가능
now = con.execute(func.sum(A.c.PK))
now.fetchall()

2023-09-06 11:35:57,298 INFO sqlalchemy.engine.Engine SELECT sum("T_A"."PK") AS sum_1 
FROM "T_A"
2023-09-06 11:35:57,299 INFO sqlalchemy.engine.Engine [generated in 0.00102s] ()


[(10,)]

In [58]:
for t in [A, B, C, D]:
    table = con.execute(t.select()) 

    for _ in table:
        print(_)

2023-09-06 11:36:05,670 INFO sqlalchemy.engine.Engine SELECT "T_A"."PK", "T_A"."NAME" 
FROM "T_A"
2023-09-06 11:36:05,671 INFO sqlalchemy.engine.Engine [generated in 0.00080s] ()
(1, '가수1')
(2, '가수2')
(3, '가수3')
(4, '가수4')
2023-09-06 11:36:05,673 INFO sqlalchemy.engine.Engine SELECT "T_B"."PK", "T_B"."NAME", "T_B"."FK" 
FROM "T_B"
2023-09-06 11:36:05,674 INFO sqlalchemy.engine.Engine [cached since 468.8s ago] ()
(1, '앨범1', 1)
(2, '앨범2', 1)
(3, '앨범1', 2)
(4, '앨범2', 2)
(5, '앨범1', 3)
(6, '앨범2', 3)
(7, '앨범1', 4)
(8, '앨범2', 4)
2023-09-06 11:36:05,676 INFO sqlalchemy.engine.Engine SELECT "T_C"."PK", "T_C"."NAME" 
FROM "T_C"
2023-09-06 11:36:05,676 INFO sqlalchemy.engine.Engine [cached since 331.8s ago] ()
(1, '장르1')
(2, '장르2')
(3, '장르3')
(4, '장르4')
2023-09-06 11:36:05,678 INFO sqlalchemy.engine.Engine SELECT "T_D"."PK", "T_D"."NAME", "T_D"."FK1", "T_D"."FK2" 
FROM "T_D"
2023-09-06 11:36:05,678 INFO sqlalchemy.engine.Engine [generated in 0.00088s] ()
(1, '1앨범_장르1_노래1', 1, 1)
(2, '1앨범_장르1_노래2'

In [59]:
# Cross_join (where문 정의)
print(select(A.c.NAME, B.c.NAME).where(A.c.PK == B.c.FK))

SELECT "T_A"."NAME", "T_B"."NAME" AS "NAME_1" 
FROM "T_A", "T_B" 
WHERE "T_A"."PK" = "T_B"."FK"


In [60]:
# Cross_join (A > B의 상속관계 사용)
print(select(A.c.NAME, B.c.NAME).join(B))

SELECT "T_A"."NAME", "T_B"."NAME" AS "NAME_1" 
FROM "T_A" JOIN "T_B" ON "T_A"."PK" = "T_B"."FK"


In [65]:
print(con.execute(A.join(B).select()))

2023-09-06 11:39:47,566 INFO sqlalchemy.engine.Engine SELECT "T_A"."PK", "T_A"."NAME", "T_B"."PK" AS "PK_1", "T_B"."NAME" AS "NAME_1", "T_B"."FK" 
FROM "T_A" JOIN "T_B" ON "T_A"."PK" = "T_B"."FK"
2023-09-06 11:39:47,568 INFO sqlalchemy.engine.Engine [generated in 0.00117s] ()


In [61]:
# Cross_join (A > B의 상속관계를 사용하되, select_from을 통해 on 절을 가져온다.)
print(select(A.c.NAME, B.c.NAME).select_from(A.join(B)))

SELECT "T_A"."NAME", "T_B"."NAME" AS "NAME_1" 
FROM "T_A" JOIN "T_B" ON "T_A"."PK" = "T_B"."FK"


In [63]:
# 3개의 결과값은 동일한 것을 확인할 수있다.
rst1 = con.execute(select(A.c.NAME, B.c.NAME).where(A.c.PK == B.c.FK)).fetchall()

rst2 = con.execute(select(A.c.NAME, B.c.NAME).join(B)).fetchall()

rst3 = con.execute(select(A.c.NAME, B.c.NAME).select_from(A.join(B))).fetchall()

print(rst1)
print(rst2)
print(rst3)

2023-09-06 11:36:27,957 INFO sqlalchemy.engine.Engine SELECT "T_A"."NAME", "T_B"."NAME" AS "NAME_1" 
FROM "T_A", "T_B" 
WHERE "T_A"."PK" = "T_B"."FK"
2023-09-06 11:36:27,957 INFO sqlalchemy.engine.Engine [cached since 14.01s ago] ()
2023-09-06 11:36:27,963 INFO sqlalchemy.engine.Engine SELECT "T_A"."NAME", "T_B"."NAME" AS "NAME_1" 
FROM "T_A" JOIN "T_B" ON "T_A"."PK" = "T_B"."FK"
2023-09-06 11:36:27,963 INFO sqlalchemy.engine.Engine [cached since 14.01s ago] ()
2023-09-06 11:36:27,963 INFO sqlalchemy.engine.Engine SELECT "T_A"."NAME", "T_B"."NAME" AS "NAME_1" 
FROM "T_A" JOIN "T_B" ON "T_A"."PK" = "T_B"."FK"
2023-09-06 11:36:27,963 INFO sqlalchemy.engine.Engine [cached since 14.01s ago] ()
[('가수1', '앨범1'), ('가수1', '앨범2'), ('가수2', '앨범1'), ('가수2', '앨범2'), ('가수3', '앨범1'), ('가수3', '앨범2'), ('가수4', '앨범1'), ('가수4', '앨범2')]
[('가수1', '앨범1'), ('가수1', '앨범2'), ('가수2', '앨범1'), ('가수2', '앨범2'), ('가수3', '앨범1'), ('가수3', '앨범2'), ('가수4', '앨범1'), ('가수4', '앨범2')]
[('가수1', '앨범1'), ('가수1', '앨범2'), ('가수2', '앨

In [66]:
rst4 = con.execute(select(A.c.PK, func.count(B.c.NAME)).\
                          select_from(A.join(B)).\
                          group_by(A.c.NAME)
                          ).fetchall()
print(rst4)

2023-09-06 11:41:04,101 INFO sqlalchemy.engine.Engine SELECT "T_A"."PK", count("T_B"."NAME") AS count_1 
FROM "T_A" JOIN "T_B" ON "T_A"."PK" = "T_B"."FK" GROUP BY "T_A"."NAME"
2023-09-06 11:41:04,101 INFO sqlalchemy.engine.Engine [generated in 0.00065s] ()
[(1, 2), (2, 2), (3, 2), (4, 2)]


In [69]:
# 가수-앨범-노래
# select_from은 PK - FK Refence가정하에 사용된다.
print(select(A.c.NAME, B.c.NAME, D.c.NAME).select_from(A.join(B.join(D))))

print(select(A.c.NAME, B.c.NAME, D.c.NAME).where((A.c.PK == B.c.FK) & (B.c.PK == D.c.FK1)))

SELECT "T_A"."NAME", "T_B"."NAME" AS "NAME_1", "T_D"."NAME" AS "NAME_2" 
FROM "T_A" JOIN ("T_B" JOIN "T_D" ON "T_B"."PK" = "T_D"."FK1") ON "T_A"."PK" = "T_B"."FK"
SELECT "T_A"."NAME", "T_B"."NAME" AS "NAME_1", "T_D"."NAME" AS "NAME_2" 
FROM "T_A", "T_B", "T_D" 
WHERE "T_A"."PK" = "T_B"."FK" AND "T_B"."PK" = "T_D"."FK1"


In [70]:
con.execute(select(A.c.NAME, B.c.NAME, D.c.NAME).select_from(A.join(B.join(D)))).fetchall()

2023-09-06 11:45:46,142 INFO sqlalchemy.engine.Engine SELECT "T_A"."NAME", "T_B"."NAME" AS "NAME_1", "T_D"."NAME" AS "NAME_2" 
FROM "T_A" JOIN ("T_B" JOIN "T_D" ON "T_B"."PK" = "T_D"."FK1") ON "T_A"."PK" = "T_B"."FK"
2023-09-06 11:45:46,143 INFO sqlalchemy.engine.Engine [generated in 0.00086s] ()


[('가수1', '앨범1', '1앨범_장르1_노래1'),
 ('가수1', '앨범1', '1앨범_장르1_노래2'),
 ('가수1', '앨범1', '1앨범_장르1_노래3'),
 ('가수1', '앨범1', '1앨범_장르1_노래4'),
 ('가수1', '앨범1', '1앨범_장르2_노래1'),
 ('가수1', '앨범1', '1앨범_장르2_노래2'),
 ('가수1', '앨범1', '1앨범_장르2_노래3'),
 ('가수1', '앨범1', '1앨범_장르2_노래4'),
 ('가수1', '앨범1', '1앨범_장르3_노래1'),
 ('가수1', '앨범1', '1앨범_장르3_노래2'),
 ('가수1', '앨범1', '1앨범_장르3_노래3'),
 ('가수1', '앨범1', '1앨범_장르3_노래4'),
 ('가수1', '앨범1', '1앨범_장르4_노래1'),
 ('가수1', '앨범1', '1앨범_장르4_노래2'),
 ('가수1', '앨범1', '1앨범_장르4_노래3'),
 ('가수1', '앨범1', '1앨범_장르4_노래4'),
 ('가수1', '앨범2', '2앨범_장르1_노래1'),
 ('가수1', '앨범2', '2앨범_장르1_노래2'),
 ('가수1', '앨범2', '2앨범_장르1_노래3'),
 ('가수1', '앨범2', '2앨범_장르1_노래4'),
 ('가수1', '앨범2', '2앨범_장르2_노래1'),
 ('가수1', '앨범2', '2앨범_장르2_노래2'),
 ('가수1', '앨범2', '2앨범_장르2_노래3'),
 ('가수1', '앨범2', '2앨범_장르2_노래4'),
 ('가수1', '앨범2', '2앨범_장르3_노래1'),
 ('가수1', '앨범2', '2앨범_장르3_노래2'),
 ('가수1', '앨범2', '2앨범_장르3_노래3'),
 ('가수1', '앨범2', '2앨범_장르3_노래4'),
 ('가수1', '앨범2', '2앨범_장르4_노래1'),
 ('가수1', '앨범2', '2앨범_장르4_노래2'),
 ('가수1', '앨범2', '2앨범_장르4_노래3'),
 ('가수1',

In [71]:
con.execute(select(A.c.NAME, B.c.NAME, D.c.NAME).where((A.c.PK == B.c.FK) & (B.c.PK == D.c.FK1))).fetchall()

2023-09-06 11:45:50,780 INFO sqlalchemy.engine.Engine SELECT "T_A"."NAME", "T_B"."NAME" AS "NAME_1", "T_D"."NAME" AS "NAME_2" 
FROM "T_A", "T_B", "T_D" 
WHERE "T_A"."PK" = "T_B"."FK" AND "T_B"."PK" = "T_D"."FK1"
2023-09-06 11:45:50,782 INFO sqlalchemy.engine.Engine [generated in 0.00172s] ()


[('가수1', '앨범1', '1앨범_장르1_노래1'),
 ('가수1', '앨범1', '1앨범_장르1_노래2'),
 ('가수1', '앨범1', '1앨범_장르1_노래3'),
 ('가수1', '앨범1', '1앨범_장르1_노래4'),
 ('가수1', '앨범1', '1앨범_장르2_노래1'),
 ('가수1', '앨범1', '1앨범_장르2_노래2'),
 ('가수1', '앨범1', '1앨범_장르2_노래3'),
 ('가수1', '앨범1', '1앨범_장르2_노래4'),
 ('가수1', '앨범1', '1앨범_장르3_노래1'),
 ('가수1', '앨범1', '1앨범_장르3_노래2'),
 ('가수1', '앨범1', '1앨범_장르3_노래3'),
 ('가수1', '앨범1', '1앨범_장르3_노래4'),
 ('가수1', '앨범1', '1앨범_장르4_노래1'),
 ('가수1', '앨범1', '1앨범_장르4_노래2'),
 ('가수1', '앨범1', '1앨범_장르4_노래3'),
 ('가수1', '앨범1', '1앨범_장르4_노래4'),
 ('가수1', '앨범2', '2앨범_장르1_노래1'),
 ('가수1', '앨범2', '2앨범_장르1_노래2'),
 ('가수1', '앨범2', '2앨범_장르1_노래3'),
 ('가수1', '앨범2', '2앨범_장르1_노래4'),
 ('가수1', '앨범2', '2앨범_장르2_노래1'),
 ('가수1', '앨범2', '2앨범_장르2_노래2'),
 ('가수1', '앨범2', '2앨범_장르2_노래3'),
 ('가수1', '앨범2', '2앨범_장르2_노래4'),
 ('가수1', '앨범2', '2앨범_장르3_노래1'),
 ('가수1', '앨범2', '2앨범_장르3_노래2'),
 ('가수1', '앨범2', '2앨범_장르3_노래3'),
 ('가수1', '앨범2', '2앨범_장르3_노래4'),
 ('가수1', '앨범2', '2앨범_장르4_노래1'),
 ('가수1', '앨범2', '2앨범_장르4_노래2'),
 ('가수1', '앨범2', '2앨범_장르4_노래3'),
 ('가수1',

In [74]:
# group by절을 넣을 수 있다.
con.execute(select(A.c.NAME, B.c.NAME, func.count(D.c.NAME)).select_from(A.join(B.join(D))).group_by(B.c.PK)).fetchall()

2023-09-06 11:47:14,676 INFO sqlalchemy.engine.Engine SELECT "T_A"."NAME", "T_B"."NAME" AS "NAME_1", count("T_D"."NAME") AS count_1 
FROM "T_A" JOIN ("T_B" JOIN "T_D" ON "T_B"."PK" = "T_D"."FK1") ON "T_A"."PK" = "T_B"."FK" GROUP BY "T_B"."PK"
2023-09-06 11:47:14,677 INFO sqlalchemy.engine.Engine [generated in 0.00077s] ()


[('가수1', '앨범1', 16),
 ('가수1', '앨범2', 16),
 ('가수2', '앨범1', 16),
 ('가수2', '앨범2', 16),
 ('가수3', '앨범1', 16),
 ('가수3', '앨범2', 16),
 ('가수4', '앨범1', 16),
 ('가수4', '앨범2', 16)]

In [76]:
# where 절을 사용해서 축약하면
# func은 DBMS에서 활용하는 함수이다 > python에서 따로 할 필요가 없다.
con.execute(select(A.c.NAME, B.c.NAME, func.count(D.c.NAME)).\
            select_from(A.join(B.join(D))).\
            where(A.c.PK == 1).\
            group_by(B.c.PK)).fetchall()

2023-09-06 11:48:50,348 INFO sqlalchemy.engine.Engine SELECT "T_A"."NAME", "T_B"."NAME" AS "NAME_1", count("T_D"."NAME") AS count_1 
FROM "T_A" JOIN ("T_B" JOIN "T_D" ON "T_B"."PK" = "T_D"."FK1") ON "T_A"."PK" = "T_B"."FK" 
WHERE "T_A"."PK" = ? GROUP BY "T_B"."PK"
2023-09-06 11:48:50,349 INFO sqlalchemy.engine.Engine [generated in 0.00122s] (1,)


[('가수1', '앨범1', 16), ('가수1', '앨범2', 16)]

In [77]:
# []로 묶으면 insert구문에서 executemany 처럼 작동한다.
con.execute(insert(A).values([{'NAME':'아무개1'}, {'NAME':'아무개2'}]))

2023-09-06 11:53:47,073 INFO sqlalchemy.engine.Engine INSERT INTO "T_A" ("NAME") VALUES (?), (?)
2023-09-06 11:53:47,074 INFO sqlalchemy.engine.Engine [no key 0.00087s] ('아무개1', '아무개2')


In [78]:
# 실제 작동 확인
con.execute(select(A)).fetchall()

2023-09-06 11:54:11,682 INFO sqlalchemy.engine.Engine SELECT "T_A"."PK", "T_A"."NAME" 
FROM "T_A"
2023-09-06 11:54:11,683 INFO sqlalchemy.engine.Engine [cached since 1086s ago] ()


[(1, '가수1'), (2, '가수2'), (3, '가수3'), (4, '가수4'), (5, '아무개1'), (6, '아무개2')]

In [79]:
print(select(A.c.NAME, B.c.NAME, D.c.NAME).select_from(A.join(B).join(D)))

SELECT "T_A"."NAME", "T_B"."NAME" AS "NAME_1", "T_D"."NAME" AS "NAME_2" 
FROM "T_A" JOIN "T_B" ON "T_A"."PK" = "T_B"."FK" JOIN "T_D" ON "T_B"."PK" = "T_D"."FK1"


In [82]:
# 가수 - 앨범 - 장르 - 노래
print(select(A.c.NAME, B.c.NAME, C.c.NAME, D.c.NAME).\
      # C는 FK가 설정되어 있지 않아서 A, B, D가 상속된 FK2의 값과 연결한다.
      select_from(A.join(B).join(D).join(C, C.c.PK == D.c.FK2)))

SELECT "T_A"."NAME", "T_B"."NAME" AS "NAME_1", "T_C"."NAME" AS "NAME_2", "T_D"."NAME" AS "NAME_3" 
FROM "T_A" JOIN "T_B" ON "T_A"."PK" = "T_B"."FK" JOIN "T_D" ON "T_B"."PK" = "T_D"."FK1" JOIN "T_C" ON "T_C"."PK" = "T_D"."FK2"


In [83]:
# Table D의 장르가 하나로 설정되어 있음. 수정필요.
con.execute(select(D)).fetchall()

2023-09-06 11:59:45,482 INFO sqlalchemy.engine.Engine SELECT "T_D"."PK", "T_D"."NAME", "T_D"."FK1", "T_D"."FK2" 
FROM "T_D"
2023-09-06 11:59:45,483 INFO sqlalchemy.engine.Engine [cached since 1420s ago] ()


[(1, '1앨범_장르1_노래1', 1, 1),
 (2, '1앨범_장르1_노래2', 1, 1),
 (3, '1앨범_장르1_노래3', 1, 1),
 (4, '1앨범_장르1_노래4', 1, 1),
 (5, '1앨범_장르2_노래1', 1, 1),
 (6, '1앨범_장르2_노래2', 1, 1),
 (7, '1앨범_장르2_노래3', 1, 1),
 (8, '1앨범_장르2_노래4', 1, 1),
 (9, '1앨범_장르3_노래1', 1, 1),
 (10, '1앨범_장르3_노래2', 1, 1),
 (11, '1앨범_장르3_노래3', 1, 1),
 (12, '1앨범_장르3_노래4', 1, 1),
 (13, '1앨범_장르4_노래1', 1, 1),
 (14, '1앨범_장르4_노래2', 1, 1),
 (15, '1앨범_장르4_노래3', 1, 1),
 (16, '1앨범_장르4_노래4', 1, 1),
 (17, '2앨범_장르1_노래1', 2, 1),
 (18, '2앨범_장르1_노래2', 2, 1),
 (19, '2앨범_장르1_노래3', 2, 1),
 (20, '2앨범_장르1_노래4', 2, 1),
 (21, '2앨범_장르2_노래1', 2, 1),
 (22, '2앨범_장르2_노래2', 2, 1),
 (23, '2앨범_장르2_노래3', 2, 1),
 (24, '2앨범_장르2_노래4', 2, 1),
 (25, '2앨범_장르3_노래1', 2, 1),
 (26, '2앨범_장르3_노래2', 2, 1),
 (27, '2앨범_장르3_노래3', 2, 1),
 (28, '2앨범_장르3_노래4', 2, 1),
 (29, '2앨범_장르4_노래1', 2, 1),
 (30, '2앨범_장르4_노래2', 2, 1),
 (31, '2앨범_장르4_노래3', 2, 1),
 (32, '2앨범_장르4_노래4', 2, 1),
 (33, '3앨범_장르1_노래1', 3, 1),
 (34, '3앨범_장르1_노래2', 3, 1),
 (35, '3앨범_장르1_노래3', 3, 1),
 (36, '3앨범_장르1_노래4', 3, 1),
 

In [84]:
meta.drop_all(engine)
meta.clear()

2023-09-06 12:01:52,413 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-06 12:01:52,415 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("T_A")
2023-09-06 12:01:52,417 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-06 12:01:52,419 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("T_B")
2023-09-06 12:01:52,420 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-06 12:01:52,421 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("T_C")
2023-09-06 12:01:52,422 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-06 12:01:52,423 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("T_D")
2023-09-06 12:01:52,423 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-06 12:01:52,423 INFO sqlalchemy.engine.Engine 
DROP TABLE "T_D"
2023-09-06 12:01:52,429 INFO sqlalchemy.engine.Engine [no key 0.00133s] ()
2023-09-06 12:01:52,432 INFO sqlalchemy.engine.Engine 
DROP TABLE "T_B"
2023-09-06 12:01:52,435 INFO sqlalchemy.engine.Engine [no key 0.00239s] ()
2023-09-06 12:01:52,437 INFO sq

In [85]:
meta.tables

FacadeDict({})

In [91]:
Table('POST', meta,
      Column('PK', Integer, primary_key= True),
      Column('CONTENT', Text),
      extend_existing = True)

Table('HASHTAG', meta,
      Column('PK', Integer, primary_key= True),
      Column('NAME', Text, nullable=False),
      # server_default는 서버에 디폴트를 저장해서 새로 만들때마다 적용한다.
      Column('Count', Integer, server_default='0'),
      extend_existing = True)

Table('POSTTAG', meta,
      Column('PFK', None, ForeignKey('POST.PK')),
      Column('HFK', Integer, ForeignKey('HASHTAG.PK')),
      extend_existing = True)


Table('POSTTAG', MetaData(), Column('PFK', Integer(), ForeignKey('POST.PK'), table=<POSTTAG>), Column('HFK', Integer(), ForeignKey('HASHTAG.PK'), table=<POSTTAG>), schema=None)

In [92]:
meta.create_all(engine)

2023-09-06 12:09:07,845 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-06 12:09:07,845 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("POST")
2023-09-06 12:09:07,857 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-06 12:09:07,858 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("POST")
2023-09-06 12:09:07,859 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-06 12:09:07,860 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("HASHTAG")
2023-09-06 12:09:07,861 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-06 12:09:07,863 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("HASHTAG")
2023-09-06 12:09:07,863 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-06 12:09:07,865 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("POSTTAG")
2023-09-06 12:09:07,865 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-06 12:09:07,865 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("POSTTAG")
2023-09-06 12:09:07,865 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-

In [94]:
A = meta.tables['POST']
B = meta.tables['HASHTAG']
C = meta.tables['POSTTAG']

In [95]:
con.execute(B.insert().values(NAME = 'A'))

2023-09-06 12:10:08,248 INFO sqlalchemy.engine.Engine INSERT INTO "HASHTAG" ("NAME") VALUES (?)
2023-09-06 12:10:08,249 INFO sqlalchemy.engine.Engine [generated in 0.00104s] ('A',)


In [96]:
con.execute(B.select()).fetchall()

2023-09-06 12:10:25,852 INFO sqlalchemy.engine.Engine SELECT "HASHTAG"."PK", "HASHTAG"."NAME", "HASHTAG"."Count" 
FROM "HASHTAG"
2023-09-06 12:10:25,853 INFO sqlalchemy.engine.Engine [generated in 0.00116s] ()


[(1, 'A', 0)]

In [116]:
con.execute(B.delete())

2023-09-06 12:15:41,299 INFO sqlalchemy.engine.Engine DELETE FROM "HASHTAG"
2023-09-06 12:15:41,300 INFO sqlalchemy.engine.Engine [cached since 293.4s ago] ()


In [169]:
con.execute(B.select()).fetchall()

2023-09-06 12:56:30,917 INFO sqlalchemy.engine.Engine SELECT "HASHTAG"."PK", "HASHTAG"."NAME", "HASHTAG"."Count" 
FROM "HASHTAG"
2023-09-06 12:56:30,918 INFO sqlalchemy.engine.Engine [cached since 2765s ago] ()


[]

In [172]:
#print(select(func.count(B.c.PK)).where(B.c.NAME == 'ORM'))
key = 'ORM'
size = con.execute(select(func.count(B.c.PK)).where(B.c.NAME == key)).fetchall()[0][0]
print(size)
if size == 0:
    con.execute(B.insert().values(NAME = key))

con.execute(select(func.count(B.c.PK)).where(B.c.NAME == key)).fetchall()[0][0]

2023-09-06 12:56:59,039 INFO sqlalchemy.engine.Engine SELECT count("HASHTAG"."PK") AS count_1 
FROM "HASHTAG" 
WHERE "HASHTAG"."NAME" = ?
2023-09-06 12:56:59,040 INFO sqlalchemy.engine.Engine [cached since 2622s ago] ('ORM',)
1
2023-09-06 12:56:59,040 INFO sqlalchemy.engine.Engine SELECT count("HASHTAG"."PK") AS count_1 
FROM "HASHTAG" 
WHERE "HASHTAG"."NAME" = ?
2023-09-06 12:56:59,041 INFO sqlalchemy.engine.Engine [cached since 2622s ago] ('ORM',)


1

In [ ]:
1. POST에 본문 insert
2. insert된 PK 가져오기
3. 위에 코드 이용해서 해시태그 PK값 가져오기
4. POSTAG에 POST.PK, HASHTAG.PK를 insert
5. HASHTAG count = HASHTAG count + 1, if PK == HASHTAG.PK 

In [126]:
con.execute(A.select())

2023-09-06 12:23:32,336 INFO sqlalchemy.engine.Engine SELECT "POST"."PK", "POST"."CONTENT" 
FROM "POST"
2023-09-06 12:23:32,337 INFO sqlalchemy.engine.Engine [cached since 10.18s ago] ()


In [176]:
con.execute(A.delete())

2023-09-06 12:57:40,801 INFO sqlalchemy.engine.Engine DELETE FROM "POST"
2023-09-06 12:57:40,801 INFO sqlalchemy.engine.Engine [cached since 1895s ago] ()


In [177]:
# 1. POST에 content 넣기
data1 = {'content': '본문1', 'tags':['Python']}

con.execute(A.insert().values(CONTENT = data1['content']))

2023-09-06 12:57:41,784 INFO sqlalchemy.engine.Engine INSERT INTO "POST" ("CONTENT") VALUES (?)
2023-09-06 12:57:41,785 INFO sqlalchemy.engine.Engine [cached since 1991s ago] ('본문1',)


In [178]:
con.execute(A.select()).fetchall()

2023-09-06 12:57:42,913 INFO sqlalchemy.engine.Engine SELECT "POST"."PK", "POST"."CONTENT" 
FROM "POST"
2023-09-06 12:57:42,914 INFO sqlalchemy.engine.Engine [cached since 2061s ago] ()


[(1, '본문1')]

In [191]:
# 2. PK값 불러오기
A_PK = con.execute(select(A.c.PK).where(A.c.CONTENT == data1['content'])).fetchall()[0]

2023-09-06 13:04:03,743 INFO sqlalchemy.engine.Engine SELECT "POST"."PK" 
FROM "POST" 
WHERE "POST"."CONTENT" = ?
2023-09-06 13:04:03,745 INFO sqlalchemy.engine.Engine [cached since 2187s ago] ('본문1',)


In [193]:
# 3. 해시태그 PK 값 매칭
#print(select(func.count(B.c.PK)).where(B.c.NAME == 'ORM'))

H_PK = con.execute(select(B.c.PK).where(B.c.PK == A_PK[0])).fetchall()[0]

2023-09-06 13:04:13,058 INFO sqlalchemy.engine.Engine SELECT "HASHTAG"."PK" 
FROM "HASHTAG" 
WHERE "HASHTAG"."PK" = ?
2023-09-06 13:04:13,058 INFO sqlalchemy.engine.Engine [cached since 728s ago] (1,)


In [194]:
H_PK

(1,)

In [206]:
# 4. POSTAG에 POST.PK, HASHTAG.PK를 insert
con.execute(insert(C).values([{'PFK':A_PK[0], 'HFK':H_PK[0]}]))

2023-09-06 13:08:24,298 INFO sqlalchemy.engine.Engine INSERT INTO "POSTTAG" ("PFK", "HFK") VALUES (?, ?)
2023-09-06 13:08:24,299 INFO sqlalchemy.engine.Engine [no key 0.00071s] (1, 1)


In [207]:
con.execute(C.select()).fetchall()

2023-09-06 13:08:25,196 INFO sqlalchemy.engine.Engine SELECT "POSTTAG"."PFK", "POSTTAG"."HFK" 
FROM "POSTTAG"
2023-09-06 13:08:25,196 INFO sqlalchemy.engine.Engine [cached since 56.33s ago] ()


[(1, 1)]

In [229]:
# 5. HASHTAG count = HASHTAG count + 1, if PK == HASHTAG.PK 
con.execute(update(B).where(B.c.PK == C.c.PFK).values({'Count': B.c.Count + 1}))

2023-09-06 13:20:12,395 INFO sqlalchemy.engine.Engine UPDATE "HASHTAG" SET "Count"=("HASHTAG"."Count" + ?) FROM "POSTTAG" WHERE "HASHTAG"."PK" = "POSTTAG"."PFK"
2023-09-06 13:20:12,400 INFO sqlalchemy.engine.Engine [generated in 0.00181s] (1,)


In [230]:
con.execute(B.select()).fetchall()

2023-09-06 13:20:14,734 INFO sqlalchemy.engine.Engine SELECT "HASHTAG"."PK", "HASHTAG"."NAME", "HASHTAG"."Count" 
FROM "HASHTAG"
2023-09-06 13:20:14,739 INFO sqlalchemy.engine.Engine [cached since 4189s ago] ()


[(1, 'ORM', 1)]

In [242]:
""" 
<HOMEWORK>
1. POST에 본문 insert
2. insert된 PK 가져오기
3. 위에 코드 이용해서 해시태그 PK값 가져오기
4. POSTAG에 POST.PK, HASHTAG.PK를 insert
5. HASHTAG count = HASHTAG count + 1, if PK == HASHTAG.PK 
 """

# engine, connect Define
engine = create_engine('sqlite:///:memory:', echo = True)
meta = MetaData()
con = engine.connect()

# 초기화
meta.drop_all(engine)
meta.clear()
meta.remove(A)
meta.remove(B)
meta.remove(C)

# 테이블 생성
Table('POST', meta,
      Column('PK', Integer, primary_key= True),
      Column('CONTENT', Text),
      extend_existing = True)

Table('HASHTAG', meta,
      Column('PK', Integer, primary_key= True),
      Column('NAME', Text, nullable=False),
      # server_default는 서버에 디폴트를 저장해서 새로 만들때마다 적용한다.
      Column('Count', Integer, server_default='0'),
      extend_existing = True)

Table('POSTTAG', meta,
      Column('PFK', None, ForeignKey('POST.PK')),
      Column('HFK', Integer, ForeignKey('HASHTAG.PK')),
      extend_existing = True)

# commit
meta.create_all(engine)

# Define Table
A = meta.tables['POST']
B = meta.tables['HASHTAG']
C = meta.tables['POSTTAG']

# 1. POST에 content 넣기 +  2. PK값 불러오기
data1 = {'content': '본문1', 'tags':['Python']}

# A에 컨탠츠를 넣음과 동시에 lastrowid로 PFK 값 추출
A_PK = con.execute(A.insert().values(CONTENT = data1['content'])).lastrowid
print(A_PK)

# 3. 해시태그 PK 값 매칭
for tag in data1['tags']:
    size = con.execute(select(func.count(B.c.Count)).where(B.c.NAME == tag)).fetchone()[0]
    if size == 0:
        con.execute(insert(B).values(NAME = tag))

H_PK = con.execute(select(B.c.PK).where(B.c.NAME == tag)).fetchall()[0]

# 4. POSTAG에 POST.PK, HASHTAG.PK를 insert
con.execute(insert(C).values([{'PFK':A_PK, 'HFK':H_PK[0]}]))

# 5. HASHTAG count = HASHTAG count + 1, if PK == HASHTAG.PK 
con.execute(update(B).where(B.c.PK == C.c.HFK).values({'Count': B.c.Count + 1}))

con.execute(select(B)).fetchall()

2023-09-07 08:01:26,054 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 08:01:26,057 INFO sqlalchemy.engine.Engine COMMIT
2023-09-07 08:01:26,062 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 08:01:26,062 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("POST")
2023-09-07 08:01:26,064 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 08:01:26,066 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("POST")
2023-09-07 08:01:26,066 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 08:01:26,066 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("HASHTAG")
2023-09-07 08:01:26,066 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 08:01:26,066 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("HASHTAG")
2023-09-07 08:01:26,066 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 08:01:26,074 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("POSTTAG")
2023-09-07 08:01:26,074 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 08:01:26,074 INFO

2023-09-07 08:01:26,084 INFO sqlalchemy.engine.Engine 
CREATE TABLE "POSTTAG" (
	"PFK" INTEGER, 
	"HFK" INTEGER, 
	FOREIGN KEY("PFK") REFERENCES "POST" ("PK"), 
	FOREIGN KEY("HFK") REFERENCES "HASHTAG" ("PK")
)


2023-09-07 08:01:26,086 INFO sqlalchemy.engine.Engine [no key 0.00063s] ()
2023-09-07 08:01:26,086 INFO sqlalchemy.engine.Engine COMMIT
2023-09-07 08:01:26,086 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 08:01:26,090 INFO sqlalchemy.engine.Engine INSERT INTO "POST" ("CONTENT") VALUES (?)
2023-09-07 08:01:26,090 INFO sqlalchemy.engine.Engine [generated in 0.00148s] ('본문1',)
1
2023-09-07 08:01:26,094 INFO sqlalchemy.engine.Engine SELECT count("HASHTAG"."Count") AS count_1 
FROM "HASHTAG" 
WHERE "HASHTAG"."NAME" = ?
2023-09-07 08:01:26,094 INFO sqlalchemy.engine.Engine [generated in 0.00107s] ('Python',)
2023-09-07 08:01:26,094 INFO sqlalchemy.engine.Engine INSERT INTO "HASHTAG" ("NAME") VALUES (?)
2023-09-07 08:01:26,094 INFO sqlalchemy.engine.Engine [generated in 

[(1, 'Python', 1)]